In [ ]:
%load_ext autoreload
%autoreload 2

import sys 
import glob
import ray
import logging 
import gc
import cv2
import time
import itertools
import pickle
import pathlib 
import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import xarray as xr
import matplotlib.gridspec as gridspec

from tqdm.notebook import tqdm
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import medfilt
from pathlib import Path
from scipy.optimize import minimize_scalar
from scipy.stats import binned_statistic


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
from kornia.geometry.transform import Affine
torch.backends.cudnn.benchmark = True

# sys.path.append(str(Path('.').absolute()))
from NLMixedUtils.utils import *
import NLMixedUtils.io_dict_to_hdf5 as ioh5
from NLMixedUtils.format_data import *
from NLMixedUtils.models import *
from NLMixedScripts.fit_GLM import *

device = torch.device("cuda:{}".format(get_freer_gpu()) if torch.cuda.is_available() else "cpu")

# pd.set_option('display.max_rows', None)
# %matplotlib widget
# ray.init(
#     ignore_reinit_error=True,
#     logging_level=logging.ERROR,
# )


In [ ]:
NKfold = 1
test_train_size = .75
frac = .1
def get_train_test_split(data,NKfold=1,test_train_size=.75,frac=.1):
    gss = GroupShuffleSplit(n_splits=NKfold, train_size=test_train_size, random_state=42)
    nT = data['model_nsp'].shape[0]
    groups = np.hstack([i*np.ones(int((frac*i)*nT) - int((frac*(i-1))*nT)) for i in range(1,int(1/frac)+1)])

    train_idx_list=[]
    test_idx_list = []
    for train_idx, test_idx in gss.split(np.arange(data.shape[0]), groups=groups):
        train_idx_list.append(train_idx)
        test_idx_list.append(test_idx)
    return train_idx_list, test_idx_list